Importing libraries

In [ ]:
%%capture
# Installing and importing required libraries
!pip install -U tweepy
!pip install -U plotly
!pip install -U requests
!pip install -U iso639
!pip install -U langdetect
import tweepy
import iso639
import requests
import tweepy
from langdetect import detect
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

%matplotlib inline

In [ ]:
#credentials from twitter developer account
API_key='yp4yfLevivSOEwgH4qLxEqaWA'
API_key_secret='MlCIza4mTIypmfx1HmAPhT8SFNJQk8Vwr8YLlaZ8iibaAqc3Sr'
Access_token='1570380778563055618-XRcQJR8x1nSrRRc2CFPiAJh4gnNZY3'
Access_token_secret='wAjHaMakwYnrwMeDk3nPZxva6ZlnOKxAvKqLDcRSFVE6h'

In [ ]:
# Authentication handler with tweepy
authenticate = tweepy.OAuthHandler(API_key,API_key_secret)
# Setting access tokens to the handler
authenticate.set_access_token(Access_token,Access_token_secret)
#defining api
API = tweepy.API(authenticate,wait_on_rate_limit=True)

For trends in India we have to get where on earth id(woeid)

In [ ]:
woeid=23424848

Get twitter Trends By WOEID

In [ ]:
def get_trends_by_location(loc_id,count):
    '''Get Trending Tweets by Location'''

    try:
        trends = API.get_place_trends(loc_id)
        df = pd.DataFrame([trending['name'],  trending['tweet_volume'], iso639.to_name(detect(trending['name']))] for trending in trends[0]['trends'])
        df.columns = ['Trends','Volume','Language']
        df = df.sort_values('Volume', ascending = True)
        return(df[:count])
    except Exception as e:
        print("An exception occurred",e)


df_india_trends = get_trends_by_location(23424848, 10)
df_india_trends

An exception occurred 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product


In [ ]:
# Setting style for slider widget
style={'description_width':'initial'}

In [ ]:
# setting limits, default value and other values for slider
limit_case = widgets.IntSlider(value=10,min=5,max=50,step=1,description='Top:',disabled=False,style=style)

In [ ]:
# defining a function to plot data
def top_n_trends(n):
    ''' Function Prints top n trending topics in India in last 24 hours, and plots out barchart for number of posts for those topics(if available on API)
    n: number of topics'''
    # creating a Dataframe for topics and volume
    df = pd.DataFrame([(j['name'],j['tweet_volume']) for j in API.get_place_trends(id=woeid)[0]['trends'][:n]],columns=(['Topic','volume']))
    # Plotting Top N topics
    px.bar(df,x='Topic',y='volume',hover_data=['Topic','volume'],color='Topic').show()

Visualize

In [ ]:
widgets.interactive(top_n_trends,n=limit_case)